<a href="https://colab.research.google.com/github/GUOJL23/COMP90089/blob/main/DataExtractionProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
project_id = 'vast-arena-431203-r0'

In [2]:
# Import libraries
from datetime import timedelta
import os

import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Make pandas dataframes prettier
from IPython.display import display, HTML, Image
%matplotlib inline

plt.style.use('ggplot')
plt.rcParams.update({'font.size': 20})

# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

In [3]:
# Authenticating to use bigquery
auth.authenticate_user()
print('Authenticated')

Authenticated


In [4]:
if project_id == 'CHANGE-ME':
  raise ValueError('You must change project_id to your GCP project.')
os.environ["GOOGLE_CLOUD_PROJECT"] = project_id

def run_query(query, project_id=project_id):
  return pd.io.gbq.read_gbq(
      query,
      project_id=project_id,
      dialect='standard')

# set the dataset
# if you want to use the demo, change this to mimic_demo
dataset = 'mimiciv'

# test it works
df = run_query("""
SELECT subject_id
FROM `physionet-data.mimiciv_hosp.patients`
WHERE subject_id = 10012853
""")
assert df.shape[0] == 1, 'unable to query MIMIC-IV!'

<ipython-input-4-3b906ba92388>:6: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


In [5]:
# Query to get itemid for creatinine in labitems table
creatinine_query = """
    SELECT itemid, label
    FROM `physionet-data.mimiciv_hosp.d_labitems`
    WHERE label LIKE '%Creatinine%';
"""
creatinine_df = run_query(creatinine_query)
creatinine_df

<ipython-input-4-3b906ba92388>:6: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


,itemid,label
0,52024,"Creatinine, Whole Blood"
1,50912,Creatinine
2,52546,Creatinine
3,51937,"Creatinine, Stool"
4,51067,24 hr Creatinine
5,51070,"Albumin/Creatinine, Urine"
6,51073,"Amylase/Creatinine Ratio, Urine"
7,51080,Creatinine Clearance
8,51081,"Creatinine, Serum"
9,51082,"Creatinine, Urine"


In [21]:
# query to get simple creatinine level data from patients
creatinine_query_icu = run_query("""
  SELECT subject_id, charttime, safe_cast(value AS FLOAT64) as creatinine_level
  FROM `physionet-data.mimiciv_hosp.labevents`
  WHERE itemid in (51081,50912,51977)
""")
creatinine_query_icu

<ipython-input-4-3b906ba92388>:6: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


KeyboardInterrupt: 

In [28]:
# Query to get subject_id, creatinine level and previous creatinine level of patients who had a creatinine level change of more that 0.3 mg/dL within 48 hrs or increased by 1.5 times in 7 days.
# info from https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10100386/
creatinine_change_query_with_icu = run_query("""
WITH creatinine_data AS (
  SELECT
    subject_id,
    charttime,
    safe_cast(value AS FLOAT64) AS creatinine_level,
    valueuom,
    LAG(safe_cast(value AS FLOAT64)) OVER (PARTITION BY subject_id ORDER BY charttime) AS previous_creatinine_level,
    DATETIME_DIFF(charttime, LAG(charttime) OVER (PARTITION BY subject_id ORDER BY charttime), HOUR) AS time_diff_hours
  FROM `physionet-data.mimiciv_hosp.labevents`
  WHERE itemid IN (51081, 50912, 51977)
)

SELECT
  subject_id,
  creatinine_level,
  previous_creatinine_level,
  (creatinine_level - previous_creatinine_level) AS creatinine_change,
  charttime,
  CASE
    WHEN (time_diff_hours <= 48 AND (creatinine_level - previous_creatinine_level) >= 0.3) THEN '0.3 mg/dL increase in 48 hours'
    WHEN (time_diff_hours <= 168 AND (creatinine_level / NULLIF(previous_creatinine_level, 0)) >= 1.5) THEN '1.5x increase in 7 days'
    ELSE 'None'
  END AS condition_satisfied
FROM creatinine_data
WHERE
  previous_creatinine_level IS NOT NULL
  AND (
    (time_diff_hours <= 48 AND (creatinine_level - previous_creatinine_level) >= 0.3)
    OR
    (time_diff_hours <= 168 AND (creatinine_level / NULLIF(previous_creatinine_level, 0)) >= 1.5)
  )

""")
creatinine_change_query_with_icu

<ipython-input-4-3b906ba92388>:6: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


,subject_id,creatinine_level,previous_creatinine_level,creatinine_change,charttime,condition_satisfied
0,10591166,8.4,7.9,0.5,2148-07-30 04:00:00,0.3 mg/dL increase in 48 hours
1,10591166,10.5,9.2,1.3,2148-09-28 10:27:00,0.3 mg/dL increase in 48 hours
2,10591166,9.2,6.9,2.3,2148-12-02 08:05:00,0.3 mg/dL increase in 48 hours
3,10591166,9.0,7.1,1.9,2149-06-05 06:00:00,0.3 mg/dL increase in 48 hours
4,10591166,11.0,9.0,2.0,2149-06-06 05:05:00,0.3 mg/dL increase in 48 hours
...,...,...,...,...,...,...
105621,19032368,1.8,1.1,0.7,2139-07-31 23:30:00,0.3 mg/dL increase in 48 hours
105622,19266431,1.8,1.5,0.3,2167-10-03 15:11:00,0.3 mg/dL increase in 48 hours
105623,19266431,1.8,1.4,0.4,2167-11-14 05:15:00,0.3 mg/dL increase in 48 hours
105624,19266431,1.8,1.4,0.4,2167-11-16 05:45:00,0.3 mg/dL increase in 48 hours


In [20]:
# Get subject id, and medication details of patients taking drugs that are found to cause aki
medication_query = run_query("""
  SELECT
    subject_id,
    hadm_id,
    drug,
    ndc,
    starttime,
    stoptime
FROM
    `physionet-data.mimiciv_hosp.prescriptions`
WHERE
    LOWER(drug) IN (
        'ibuprofen', 'naproxen', 'aspirin', 'diclofenac', 'indomethacin',
        'gentamicin', 'tobramycin', 'vancomycin', 'amphotericin b', 'trimethoprim-sulfamethoxazole',
        'cephalexin', 'furosemide', 'bumetanide', 'hydrochlorothiazide', 'spironolactone',
        'amiloride', 'lisinopril', 'enalapril', 'ramipril', 'captopril',
        'losartan', 'valsartan', 'irbesartan', 'candesartan', 'cisplatin',
        'carboplatin', 'methotrexate', 'ifosfamide', 'cyclosporine', 'tacrolimus',
        'sirolimus', 'acyclovir', 'foscarnet', 'tenofovir', 'cidofovir',
        'omeprazole', 'pantoprazole', 'lansoprazole', 'esomeprazole', 'allopurinol',
        'lithium'
    )
ORDER BY
    subject_id, starttime
LIMIT 1000
""")
medication_query

<ipython-input-4-3b906ba92388>:6: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


,subject_id,hadm_id,drug,ndc,starttime,stoptime
0,10000032,22595853,Spironolactone,63739054410,2180-05-07 01:00:00,2180-05-07 09:00:00
1,10000032,22595853,Furosemide,51079007220,2180-05-07 01:00:00,2180-05-07 09:00:00
2,10000032,22595853,Spironolactone,63739054410,2180-05-08 08:00:00,2180-05-07 22:00:00
3,10000032,22595853,Furosemide,51079007320,2180-05-08 08:00:00,2180-05-07 22:00:00
4,10000032,22841357,Furosemide,51079007220,2180-06-26 23:00:00,2180-06-26 22:00:00
...,...,...,...,...,...,...
995,10008924,27441295,Furosemide,00182117089,2139-04-17 20:00:00,2139-04-18 09:00:00
996,10008924,27441295,Pantoprazole,00008084199,2139-04-18 16:00:00,2139-04-22 19:00:00
997,10008924,27441295,Furosemide,00182116189,2139-04-18 20:00:00,2139-04-22 19:00:00
998,10008924,23676183,Omeprazole,00093521193,2139-07-08 20:00:00,2139-07-09 23:00:00


In [32]:
# Medication details of patinets who satify aki criteria using result of last query
subject_ids = tuple(creatinine_change_query_with_icu['subject_id'].unique())

# New query to fetch medication details based on subject_id from previous query
medication_query_with_creatinine_change = run_query(f"""
SELECT
  subject_id,
  hadm_id,
  drug,
  ndc,
  starttime,
  stoptime
FROM
  `physionet-data.mimiciv_hosp.prescriptions`
WHERE
  subject_id IN {subject_ids}
  AND LOWER(drug) IN (
    'ibuprofen', 'naproxen', 'aspirin', 'diclofenac', 'indomethacin',
    'gentamicin', 'tobramycin', 'vancomycin', 'amphotericin b', 'trimethoprim-sulfamethoxazole',
    'cephalexin', 'furosemide', 'bumetanide', 'hydrochlorothiazide', 'spironolactone',
    'amiloride', 'lisinopril', 'enalapril', 'ramipril', 'captopril',
    'losartan', 'valsartan', 'irbesartan', 'candesartan', 'cisplatin',
    'carboplatin', 'methotrexate', 'ifosfamide', 'cyclosporine', 'tacrolimus',
    'sirolimus', 'acyclovir', 'foscarnet', 'tenofovir', 'cidofovir',
    'omeprazole', 'pantoprazole', 'lansoprazole', 'esomeprazole', 'allopurinol',
    'lithium'
  )
ORDER BY
  subject_id, starttime
""")

<ipython-input-4-3b906ba92388>:6: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


In [36]:
# Assuming medication_query_with_creatinine_change contains the new medication data
medication_data = medication_query_with_creatinine_change

# Merge the creatinine change data with medication data
merged_data = pd.merge(
    medication_data,
    creatinine_change_query_with_icu,
    on='subject_id',  # You can join on subject_id
    how='inner'
)

# Ensure that 'charttime' and 'starttime' are in datetime format
merged_data['charttime'] = pd.to_datetime(merged_data['charttime'])
merged_data['starttime'] = pd.to_datetime(merged_data['starttime'])

# Calculate the difference in days and add a new column 'days_prescribed_ahead'
merged_data['days_prescribed_ahead'] = (merged_data['charttime'] - merged_data['starttime']).dt.days

# Filter medications prescribed before the creatinine charttime
filtered_data = merged_data[merged_data['starttime'] < merged_data['charttime']]

In [37]:
filtered_data

,subject_id,hadm_id,drug,ndc,starttime,stoptime,creatinine_level,previous_creatinine_level,creatinine_change,charttime,condition_satisfied,days_prescribed_ahead
0,10000032,22595853,Furosemide,51079007220,2180-05-07 01:00:00,2180-05-07 09:00:00,0.6,0.3,0.3,2180-06-26 16:10:00,1.5x increase in 7 days,50.0
1,10000032,22595853,Furosemide,51079007220,2180-05-07 01:00:00,2180-05-07 09:00:00,0.8,0.4,0.4,2180-08-10 12:00:00,1.5x increase in 7 days,95.0
2,10000032,22595853,Spironolactone,63739054410,2180-05-07 01:00:00,2180-05-07 09:00:00,0.6,0.3,0.3,2180-06-26 16:10:00,1.5x increase in 7 days,50.0
3,10000032,22595853,Spironolactone,63739054410,2180-05-07 01:00:00,2180-05-07 09:00:00,0.8,0.4,0.4,2180-08-10 12:00:00,1.5x increase in 7 days,95.0
4,10000032,22595853,Furosemide,51079007320,2180-05-08 08:00:00,2180-05-07 22:00:00,0.6,0.3,0.3,2180-06-26 16:10:00,1.5x increase in 7 days,49.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5797582,19998497,28129567,Aspirin,00904404073,2145-02-26 08:00:00,2145-02-28 16:00:00,2.4,1.7,0.7,2145-02-28 07:40:00,0.3 mg/dL increase in 48 hours,1.0
5797583,19998497,28129567,Aspirin,00904404073,2145-02-26 08:00:00,2145-02-28 16:00:00,2.2,1.8,0.4,2145-07-30 06:15:00,0.3 mg/dL increase in 48 hours,153.0
5797586,19998497,21557581,Furosemide,51079007220,2145-07-25 02:00:00,2145-08-01 17:00:00,2.2,1.8,0.4,2145-07-30 06:15:00,0.3 mg/dL increase in 48 hours,5.0
5797589,19998497,21557581,Aspirin,00904404073,2145-07-25 10:00:00,2145-08-01 17:00:00,2.2,1.8,0.4,2145-07-30 06:15:00,0.3 mg/dL increase in 48 hours,4.0


In [45]:
demographics_query = run_query(f"""
SELECT
    subject_id,
    gender,
    anchor_age
FROM
    `physionet-data.mimiciv_hosp.patients`
WHERE
    subject_id IN {subject_ids}
ORDER BY
    subject_id
""")
demographics_query

<ipython-input-4-3b906ba92388>:6: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


,subject_id,gender,anchor_age
0,10000032,F,52
1,10000935,F,52
2,10000980,F,73
3,10001338,F,43
4,10002013,F,53
...,...,...,...
28863,19997538,M,53
28864,19997752,F,66
28865,19998330,F,71
28866,19998497,F,82


In [46]:
merged_data = pd.merge(
    filtered_data,           # Use the DataFrame that contains medication and creatinine change data
    demographics_query,      # New demographics data
    on='subject_id',        # Join on subject_id
    how='left'              # Use left join to keep all entries from filtered_data
)
merged_data

,subject_id,hadm_id,drug,ndc,starttime,stoptime,creatinine_level,previous_creatinine_level,creatinine_change,charttime,condition_satisfied,days_prescribed_ahead,gender,anchor_age
0,10000032,22595853,Furosemide,51079007220,2180-05-07 01:00:00,2180-05-07 09:00:00,0.6,0.3,0.3,2180-06-26 16:10:00,1.5x increase in 7 days,50.0,F,52
1,10000032,22595853,Furosemide,51079007220,2180-05-07 01:00:00,2180-05-07 09:00:00,0.8,0.4,0.4,2180-08-10 12:00:00,1.5x increase in 7 days,95.0,F,52
2,10000032,22595853,Spironolactone,63739054410,2180-05-07 01:00:00,2180-05-07 09:00:00,0.6,0.3,0.3,2180-06-26 16:10:00,1.5x increase in 7 days,50.0,F,52
3,10000032,22595853,Spironolactone,63739054410,2180-05-07 01:00:00,2180-05-07 09:00:00,0.8,0.4,0.4,2180-08-10 12:00:00,1.5x increase in 7 days,95.0,F,52
4,10000032,22595853,Furosemide,51079007320,2180-05-08 08:00:00,2180-05-07 22:00:00,0.6,0.3,0.3,2180-06-26 16:10:00,1.5x increase in 7 days,49.0,F,52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3369127,19998497,28129567,Aspirin,00904404073,2145-02-26 08:00:00,2145-02-28 16:00:00,2.4,1.7,0.7,2145-02-28 07:40:00,0.3 mg/dL increase in 48 hours,1.0,F,82
3369128,19998497,28129567,Aspirin,00904404073,2145-02-26 08:00:00,2145-02-28 16:00:00,2.2,1.8,0.4,2145-07-30 06:15:00,0.3 mg/dL increase in 48 hours,153.0,F,82
3369129,19998497,21557581,Furosemide,51079007220,2145-07-25 02:00:00,2145-08-01 17:00:00,2.2,1.8,0.4,2145-07-30 06:15:00,0.3 mg/dL increase in 48 hours,5.0,F,82
3369130,19998497,21557581,Aspirin,00904404073,2145-07-25 10:00:00,2145-08-01 17:00:00,2.2,1.8,0.4,2145-07-30 06:15:00,0.3 mg/dL increase in 48 hours,4.0,F,82


In [44]:
race_query = run_query(f"""
SELECT
    distinct(subject_id),
    race
FROM
    `physionet-data.mimiciv_hosp.admissions`
WHERE
    subject_id IN {subject_ids}
ORDER BY
    subject_id
""")
race_query

<ipython-input-4-3b906ba92388>:6: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


,subject_id,race
0,10000032,WHITE
1,10000935,BLACK/AFRICAN AMERICAN
2,10000980,BLACK/AFRICAN AMERICAN
3,10001338,WHITE
4,10002013,WHITE
...,...,...
31499,19997538,WHITE
31500,19997752,WHITE
31501,19998330,BLACK/AFRICAN AMERICAN
31502,19998497,WHITE


In [47]:
merged_data = pd.merge(
    filtered_data,           # Use the DataFrame that contains medication and creatinine change data
    race_query,      # New demographics data
    on='subject_id',        # Join on subject_id
    how='left'              # Use left join to keep all entries from filtered_data
)
merged_data

,subject_id,hadm_id,drug,ndc,starttime,stoptime,creatinine_level,previous_creatinine_level,creatinine_change,charttime,condition_satisfied,days_prescribed_ahead,race
0,10000032,22595853,Furosemide,51079007220,2180-05-07 01:00:00,2180-05-07 09:00:00,0.6,0.3,0.3,2180-06-26 16:10:00,1.5x increase in 7 days,50.0,WHITE
1,10000032,22595853,Furosemide,51079007220,2180-05-07 01:00:00,2180-05-07 09:00:00,0.8,0.4,0.4,2180-08-10 12:00:00,1.5x increase in 7 days,95.0,WHITE
2,10000032,22595853,Spironolactone,63739054410,2180-05-07 01:00:00,2180-05-07 09:00:00,0.6,0.3,0.3,2180-06-26 16:10:00,1.5x increase in 7 days,50.0,WHITE
3,10000032,22595853,Spironolactone,63739054410,2180-05-07 01:00:00,2180-05-07 09:00:00,0.8,0.4,0.4,2180-08-10 12:00:00,1.5x increase in 7 days,95.0,WHITE
4,10000032,22595853,Furosemide,51079007320,2180-05-08 08:00:00,2180-05-07 22:00:00,0.6,0.3,0.3,2180-06-26 16:10:00,1.5x increase in 7 days,49.0,WHITE
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4169132,19998497,28129567,Aspirin,00904404073,2145-02-26 08:00:00,2145-02-28 16:00:00,2.4,1.7,0.7,2145-02-28 07:40:00,0.3 mg/dL increase in 48 hours,1.0,WHITE
4169133,19998497,28129567,Aspirin,00904404073,2145-02-26 08:00:00,2145-02-28 16:00:00,2.2,1.8,0.4,2145-07-30 06:15:00,0.3 mg/dL increase in 48 hours,153.0,WHITE
4169134,19998497,21557581,Furosemide,51079007220,2145-07-25 02:00:00,2145-08-01 17:00:00,2.2,1.8,0.4,2145-07-30 06:15:00,0.3 mg/dL increase in 48 hours,5.0,WHITE
4169135,19998497,21557581,Aspirin,00904404073,2145-07-25 10:00:00,2145-08-01 17:00:00,2.2,1.8,0.4,2145-07-30 06:15:00,0.3 mg/dL increase in 48 hours,4.0,WHITE


In [38]:
diagnosis_query = run_query("""
SELECT
    subject_id,
    hadm_id,
    icd_code,
    seq_num
FROM
`physionet-data.mimiciv_hosp.diagnoses_icd`
WHERE
    (icd_version = 10 AND icd_code IN ('N170', 'N171', 'N172', 'N178', 'N179'))
ORDER BY
    subject_id, hadm_id
LIMIT 1000;
""")
diagnosis_query

<ipython-input-4-3b906ba92388>:6: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


,subject_id,hadm_id,icd_code,seq_num
0,10001401,26840593,N179,3
1,10001401,27012892,N179,4
2,10001401,27060146,N179,4
3,10001884,24746267,N179,2
4,10002013,25442395,N179,4
...,...,...,...,...
995,10444108,29709801,N179,3
996,10444484,23030871,N179,7
997,10445331,24253259,N179,8
998,10446424,20299462,N179,2


In [ ]:
# File path to save the CSV file
csv_file_path = 'final_patient_data.csv'

# Save the final_result DataFrame to a CSV file
merged_data.to_csv(csv_file_path, index=False)

# Optionally, you can print a confirmation message
print(f"Final result saved to {csv_file_path}")